In [1]:
! pip install pymongo transformers datasets faiss-gpu pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found 

In [2]:
import pandas as pd
df = pd.read_csv('candidates.csv')
df

,Name,Contact Details,Location,Job Skills,Experience,Projects,Comments
0,John Doe,john.doe@example.com,New York,Java; Big Data; Hadoop,5 years,Developed a big data processing system using H...,Strong problem-solving skills.
1,Jane Smith,jane.smith@example.com,San Francisco,JavaScript; React; Node.js,3 years,Built a real-time chat application using React...,Excellent in team collaboration.
2,Bob Johnson,bob.johnson@example.com,New York,Java; Spring; Microservices,6 years,Implemented a microservices architecture for a...,Proactive and detail-oriented.
3,Alice Brown,alice.brown@example.com,Chicago,Python; Django; Machine Learning,4 years,Created a machine learning model for predictiv...,Strong analytical skills.
4,Michael Green,michael.green@example.com,Boston,Ruby; Rails; PostgreSQL,7 years,Developed a scalable web application using Rub...,Great leadership qualities.
...,...,...,...,...,...,...,...
115,Nancy Drew,nancy.drew@gmail.com,"New York, NY","Spring Boot, Node.js, Ruby, Java",2 years,Front-end redesign,Machine learning expert
116,Charlie Brown,charlie.brown@outlook.com,"Philadelphia, PA","Flask, AWS, Machine Learning, Go",10 years,Microservices architecture,Expert in backend development
117,Quinn Fabray,quinn.fabray@gmail.com,"Los Angeles, CA","Django, Microservices, Machine Learning, Azure",14 years,API development,Excellent team player
118,Daisy Johnson,daisy.johnson@outlook.com,"San Diego, CA","Node.js, Kubernetes, React, Django",2 years,Microservices architecture,Front-end specialist


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

In [5]:
data=pd.read_csv('Resume.csv')
data.head()

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR


In [5]:
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
tokenized_inputs = tokenizer(list(data["Resume_str"]), padding=True, truncation=True, return_tensors="pt")

In [8]:
categories = data["Category"].unique()
category_to_id = {category: idx for idx, category in enumerate(categories)}
labels = data["Category"].map(category_to_id).values

In [9]:
class ResumeDataset(torch.utils.data.Dataset):
    def __init__(self, tokenized_inputs, labels):
        self.input_ids = tokenized_inputs["input_ids"]
        self.attention_mask = tokenized_inputs["attention_mask"]
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.labels[idx],
        }

In [10]:
dataset = ResumeDataset(tokenized_inputs, labels)

In [11]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=len(categories))

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
)

trainer.train()

model.save_pretrained("fine-tuned-model")
tokenizer.save_pretrained("fine-tuned-model")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,1.148200
2,1.761300,0.678456
3,1.761300,0.572478


('fine-tuned-model/tokenizer_config.json',
 'fine-tuned-model/special_tokens_map.json',
 'fine-tuned-model/vocab.txt',
 'fine-tuned-model/added_tokens.json',
 'fine-tuned-model/tokenizer.json')

In [6]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('/content/fine-tuned-model')
model = AutoModelForSequenceClassification.from_pretrained('/content/fine-tuned-model')

In [15]:
def create_profile_text(row):
    return f"{row['Job Skills']}. {row['Experience']}. {row['Projects']}"

df['Profile Text'] = df.apply(create_profile_text, axis=1)

In [16]:
df.head()

,Name,Contact Details,Location,Job Skills,Experience,Projects,Comments,Profile Text
0,John Doe,john.doe@example.com,New York,Java; Big Data; Hadoop,5 years,Developed a big data processing system using H...,Strong problem-solving skills.,Java; Big Data; Hadoop. 5 years. Developed a b...
1,Jane Smith,jane.smith@example.com,San Francisco,JavaScript; React; Node.js,3 years,Built a real-time chat application using React...,Excellent in team collaboration.,JavaScript; React; Node.js. 3 years. Built a r...
2,Bob Johnson,bob.johnson@example.com,New York,Java; Spring; Microservices,6 years,Implemented a microservices architecture for a...,Proactive and detail-oriented.,Java; Spring; Microservices. 6 years. Implemen...
3,Alice Brown,alice.brown@example.com,Chicago,Python; Django; Machine Learning,4 years,Created a machine learning model for predictiv...,Strong analytical skills.,Python; Django; Machine Learning. 4 years. Cre...
4,Michael Green,michael.green@example.com,Boston,Ruby; Rails; PostgreSQL,7 years,Developed a scalable web application using Rub...,Great leadership qualities.,Ruby; Rails; PostgreSQL. 7 years. Developed a ...


In [31]:
import pandas as pd
import json

# Load the CSV file
csv_file_path = '/content/candidates.csv'
df = pd.read_csv(csv_file_path)

# Convert DataFrame to JSON
candidates_data = df.to_dict(orient='records')


In [33]:
! pip install faiss-gpu sentence-transformers pandas

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [36]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load the sentence transformer model
model = SentenceTransformer('/content/fine-tuned-model')

# Function to generate embeddings
def embed_text(text):
    return model.encode(text)

# Generate embeddings for candidate profiles
embeddings = []
metadata = []
for record in candidates_data:
    profile_text = f"{record['Name']} {record['Contact Details']} {record['Location']} {record['Job Skills']} {record['Experience']} {record['Projects']} {record['Comments']}"
    embedding = embed_text(profile_text)
    embeddings.append(embedding)
    metadata.append(record)

embeddings = np.array(embeddings).astype('float32')

# Create FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Function to get top N profiles
def get_top_n_profiles(job_description, n=10):
    job_embedding = embed_text(job_description).reshape(1, -1)
    distances, indices = index.search(job_embedding, n)
    return [metadata[i] for i in indices[0]]

# Command line interface for querying
job_description = input("question")
profiles = get_top_n_profiles(job_description)
print("Top matching candidate profiles:")
for profile in profiles:
    print(profile['Name'])


questionPick up the top 10 profiles for the following job description, We are looking for a skilled UI Developer to join our dynamic team. The ideal candidate will have a strong background in front-end development, with proficiency in HTML, CSS, JavaScript, and modern frameworks like React or Angular. Your primary responsibility will be to create visually appealing and user-friendly web interfaces that enhance user experience and align with our brand guidelines.
Top matching candidate profiles:
Laura Davis
Quinn Fabray
Michael Scott
Rachel Green
Lily Adams
Nancy Lewis
Kyle Reese
Nancy Drew
Jane Smith
Daniel Clark
